In [1]:
import os, glob, json, csv, subprocess, sys, re
from git import *
from subprocess import Popen, PIPE
from os import path
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objs as go
import plotly.plotly as ply
import plotly
from plotly.offline import plot, init_notebook_mode, iplot

%matplotlib inline

# Defining repository

In [2]:
userhome = os.path.expanduser('~')
repository = os.path.dirname(userhome + r'/historage-mining/historage/hbase/')

# Comparing files based on the number of line changed

Finding files with different number of line changed

In [23]:
cols = ['filename','SUM #line_changed']
datahist = pd.read_csv('diffhistogramfilechanged.csv')
datamyers = pd.read_csv('diffmyersfilechanged.csv')
datamin = pd.read_csv('diffminimalfilechanged.csv')
datapat = pd.read_csv('diffpatiencefilechanged.csv')
dfhist = datahist[cols]
dfmyers = datamyers[cols]
dfmin = datamin[cols]
dfpat = datapat[cols]

In [24]:
lshist = []
for x in range(0,len(dfhist)-1):
    lshist.append(list(dfhist.iloc[x]))

In [25]:
lsmyers = []
for y in range(0,len(dfmyers)-1):
    lsmyers.append(list(dfmyers.iloc[y]))

In [26]:
lsmin = []
for z in range(0,len(dfmin)-1):
    lsmin.append(list(dfmin.iloc[z]))

In [27]:
lspat = []
for w in range(0,len(dfpat)-1):
    lspat.append(list(dfpat.iloc[w]))

In [28]:
datahist = datahist.sort_values('filename')
datamyers = datamyers.sort_values('filename')
datamin = datamin.sort_values('filename')
datapat = datapat.sort_values('filename')
len(datahist)

16217

In [37]:
#Merge dataframes hist and myers
datamerge1 = datahist.merge(datamyers, on="filename", how="outer", suffixes=('-hist', '-myers'))
#Merge dataframes min and pat
datamerge2 = datamin.merge(datapat, on="filename", how="outer", suffixes=('-min', '-pat'))
#Merge all dataframes
datamerge3 = datamerge1.merge(datamerge2, on="filename", how="outer")

#Remove the rows with same number of changes between hist and myers
#datamerge3 = datamerge3[datamerge3['SUM #line_changed-hist'] != datamerge3['SUM #line_changed-myers']]

#Fill NAs with 0 and sort by filename
datamerge3.fillna(0, inplace=True)
datamerge3 = datamerge3.sort_values(by ='SUM #line_changed-hist', ascending=False).reset_index(drop=True)
colnames = ['filename','SUM #line_changed-hist','SUM #line_changed-myers','SUM #line_changed-min','SUM #line_changed-pat']
datamerge3 = datamerge3[colnames]
datamerge3.to_csv('diff_file.csv')
datamerge3.head()

,filename,SUM #line_changed-hist,SUM #line_changed-myers,SUM #line_changed-min,SUM #line_changed-pat
0,.../hbase/protobuf/generated/MasterProtos.java,446784.0,582362.0,446760.0,446796.0
1,.../shaded/protobuf/generated/MasterProtos.java,365418.0,374934.0,365406.0,365442.0
2,.../hbase/protobuf/generated/ClientProtos.java,317592.0,380722.0,312192.0,313424.0
3,.../hadoop/hbase/thrift/generated/Hbase.java,275128.0,283020.0,274488.0,274816.0
4,.../hbase/thrift2/generated/THBaseService.java,252425.0,252811.0,252391.0,252425.0


In [38]:
df = pd.read_csv('diff_file.csv')
#df = df.sort_values('SUM #line_changed-hist', ascending=False)

trace1 = go.Scatter(
    x=df['filename'][:10],
    y=df['SUM #line_changed-myers'][:10],
    name='--diff-algorithm=myers'
)

trace2 = go.Scatter(
    x=df['filename'][:10],
    y=df['SUM #line_changed-hist'][:10],
    name='--diff-algorithm=histogram'
)

trace3 = go.Scatter(
    x=df['filename'][:10],
    y=df['SUM #line_changed-min'][:10],
    name='--diff-algorithm=minimal'
)

trace4 = go.Scatter(
    x=df['filename'][:10],
    y=df['SUM #line_changed-pat'][:10],
    name='--diff-algorithm=patience'
)

dt = [trace1, trace2, trace3, trace4]
layout = go.Layout(
    barmode='group'
)

init_notebook_mode(connected=True)
fig = go.Figure(data=dt, layout=layout)
iplot(fig, show_link=False)